# Replicating Jorion (2003)
Portfolio optimization with tracking error constraints

In [1]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# benchmark weights in N asset classes
q = np.array([0.25, 0.25, 0.25, 0.25])

In [7]:
# deviations from the benchmark (have to sum to zero!)
x = np.array([-0.05, 0.1 ,-0.05, 0])

In [9]:
# portfolio asset class weights
pf = q + x

In [23]:
E = np.array([0.03, 0.1, 0.05, 0.08])

In [42]:
# define correlation matrix, and vector of risk per asset class, which is then used to get the matrix..
rho = np.array([
        [1.0   ,0.1  ,0.4, -0.2]
        ,[0.1  ,1.0  ,0.5, 0.2]
        ,[0.4  ,0.5  ,1.0, 0.6]
        ,[-0.2 ,0.2  ,0.6, 1.0]
    ])
sigma = [0.03, 0.15, 0.1, 0.08]

In [43]:
def corr2cov(sigma, corr):
    return corr * np.outer(sigma, sigma)

In [44]:
# Variance-covariance matrix
V = corr2cov(sigma, rho)

In [45]:
V

array([[ 0.0009 ,  0.00045,  0.0012 , -0.00048],
       [ 0.00045,  0.0225 ,  0.0075 ,  0.0024 ],
       [ 0.0012 ,  0.0075 ,  0.01   ,  0.0048 ],
       [-0.00048,  0.0024 ,  0.0048 ,  0.0064 ]])

In [61]:
# Expected return on the benchmark
np.dot(q, E)

0.065000000000000002

In [62]:
# absolute volatility of benchmark
np.dot(np.dot(q, V), q)

0.0044712499999999995

In [63]:
# relative return
np.dot(x, E)

0.0060000000000000019

In [64]:
# Tracking error
np.dot(np.dot(x, V), x)

0.00017875000000000001

In [65]:
# actual portfolio return
np.dot(pf, E)

0.070999999999999994

In [68]:
# actual portfolio risk
np.dot(np.dot(pf, V), pf)

0.0056532500000000003

In [ ]:
# add check that the portfolio weights sum to unity
# and add check the relative positions sum to zero